<a href="https://colab.research.google.com/github/Mahalia-ym/entregaTFT/blob/main/python/clasificador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Se importan las librerías necesarias para realizar la red neuronal


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import xgboost as xgb

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

Se da acceso al drive donde se encuentran los datos de los constructos que vamos a utilizar como input para la red neuronal

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Aquí se cargan los valores de los constructos

In [ ]:
datosCompletos = pd.read_csv('/content/drive/My Drive/2º Semestre/TFT Nuevo/Red Neuronal/constructScores.csv', encoding='utf-8', sep=',', index_col=0)
datosCompletos.sample(10)


,Mindfulness,Satisf,Performance,Abandon
24,-0.489036,-0.772308,-0.317626,1.086994
192,0.323871,0.896895,-0.799480,-0.501475
122,-1.236202,-0.774597,-1.281334,1.100374
112,-1.428787,0.387285,0.952107,-1.405734
129,-1.236202,0.898040,-0.164128,-0.501475
171,0.938309,0.551497,-0.935152,-0.294805
140,2.091928,1.594658,1.587459,-1.405734
63,-1.983367,-1.379759,-0.606796,1.784583
18,0.895199,0.686660,-0.799480,-0.963742
33,-2.787218,0.162742,1.280463,1.100374


Iniciamos el modelo con el clasificador elegido

In [ ]:
modeloXGB = xgb.XGBRegressor(objective='reg:squarederror', colsamplebytree=0.5,
                             learning_rate=0.1, max_depth=5, alpha=5, n_estimators=500)

Establecemos la mejor estructura neuronal del anterior programa

In [ ]:
modelo=Sequential()
#Capa entrada
modelo.add(Dense(3, activation='relu', input_shape=(3,)))
#intermedias
modelo.add(Dense(8, activation='relu'))
modelo.add(Dense(3, activation='relu'))
#salida
modelo.add(Dense(1, activation='linear'))

In [ ]:
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 12        
                                                                 
 dense_1 (Dense)             (None, 8)                 32        
                                                                 
 dense_2 (Dense)             (None, 3)                 27        
                                                                 
 dense_3 (Dense)             (None, 1)                 4         
                                                                 
Total params: 75
Trainable params: 75
Non-trainable params: 0
_________________________________________________________________


Establecemos dos variables las cuales sacarán los valores RMSE y R^2 de cada iteración según el mejor KFolder elgido anteriormente. Se ejecutará la predicción del modelo a través del clasificador para poder calcular las variables de medida.

In [ ]:
precision = []
precision2 = []
calculate_kf = KFold(n_splits=7, shuffle=False)
optimizador=tf.keras.optimizers.Adam(0.01)
#modelo.compile(optimizer=optimizador, loss='mean_squared_error')
resultado= []
#Vamos a dividir los valores del kf
for valEntreno, valPrueba in calculate_kf.split(datosCompletos):
  entrenoX = datosCompletos.iloc[valEntreno].loc[:, ["Mindfulness", "Satisf", "Performance"]]
  pruebaX = datosCompletos.iloc[valPrueba].loc[:, ["Mindfulness", "Satisf", "Performance"]]
  entrenoY = datosCompletos.iloc[valEntreno].loc[:, ["Abandon"]]
  pruebaY = datosCompletos.iloc[valPrueba].loc[:, ["Abandon"]]
  modeloXGB.fit(entrenoX, entrenoY)
  pred_y = modeloXGB.predict(pruebaX)
  #resultado = pred_y
  #print("mse")
  precision.append(mean_squared_error(pruebaY, pred_y, squared=False))
  #print(precision)

  #print("*********************")
  #print("R^2")
  precision2.append(r2_score(pruebaY, pred_y))
  #print(precision2)

print("mse")
print(precision)
print("***********************")
print("R^2")
print(precision2)


Calculamos la media de las medidas RMSE y R^2

In [ ]:
mse = 0
for i in precision:
  mse += i
print("Media mse:")
print(mse/7)

r2 = 0
for j in precision2:
  r2 += j
print("R^2:")
print(r2/7)